In [1]:
# 💹 Indian Stock Market 10-Year Beginner-Friendly Investment Advisor
# Focus: BLUE-CHIP Stocks (Top 5)
import warnings
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import yfinance as yf

warnings.filterwarnings("ignore", category=FutureWarning)

START_DATE = (datetime.today() - timedelta(days=365 * 10)).strftime('%Y-%m-%d')
END_DATE = datetime.today().strftime('%Y-%m-%d')
RISK_FREE_RATE = 0.05

# Blue-Chip Universe (Reliable, large, stable companies)
BLUECHIP = [
    "ADANIENT","ADANIPORTS","ASIANPAINT","AXISBANK","BAJAJ-AUTO","BAJAJFINSV","BAJFINANCE",
    "BHARTIARTL","BPCL","BRITANNIA","CIPLA","COALINDIA","DIVISLAB","DRREDDY","EICHERMOT",
    "GRASIM","HCLTECH","HDFCBANK","HDFCLIFE","HEROMOTOCO","HINDALCO","HINDUNILVR",
    "ICICIBANK","INDUSINDBK","INFY","ITC","JSWSTEEL","KOTAKBANK","LT","M&M","MARUTI",
    "NESTLEIND","NTPC","ONGC","POWERGRID","RELIANCE","SBILIFE","SBIN","SUNPHARMA",
    "TATACONSUM","TATAMOTORS","TATASTEEL","TCS","TECHM","TITAN","ULTRACEMCO","UPL","WIPRO"
]

def download_adj_close(tickers):
    tickers_ns = [t + ".NS" for t in tickers]
    print(f"\n📥 Downloading {len(tickers)} Blue-Chip tickers... This may take a few minutes.")
    raw = yf.download(tickers_ns, start=START_DATE, end=END_DATE, progress=False, group_by="ticker", threads=False)
    adj = pd.DataFrame()
    for t in tickers_ns:
        sym = t.replace(".NS", "")
        try:
            if "Adj Close" in raw[t].columns:
                adj[sym] = raw[t]["Adj Close"]
            elif "Close" in raw[t].columns:
                adj[sym] = raw[t]["Close"]
            else:
                adj[sym] = raw.filter(like=sym).iloc[:, 0]
        except Exception:
            try:
                adj[sym] = raw.filter(like=sym).iloc[:, 0]
            except Exception:
                pass
    adj = adj.ffill().bfill()
    adj = adj.dropna(axis=1, thresh=int(0.80 * len(adj)))
    return adj

def compute_cagr(series):
    s = series.dropna()
    if s.empty: return np.nan
    years = (s.index[-1] - s.index[0]).days / 365.25
    return (s.iloc[-1] / s.iloc[0]) ** (1 / years) - 1

def downside_std(returns):
    neg = returns[returns < 0]
    return neg.std() if not neg.empty else 0

def rolling_recovery_days(price):
    s = price.dropna()
    if s.empty: return np.nan
    peak, last_peak = s.iloc[0], s.index[0]
    max_days = 0
    for date, val in s.items():
        if val >= peak:
            peak = val
            last_peak = date
        else:
            days_under = (date - last_peak).days
            if days_under > max_days:
                max_days = days_under
    return max_days

# METRIC CALCULATION
def calculate_metrics(adj):
    returns = adj.pct_change().dropna()
    nifty_ref = yf.download("^NSEI", start=START_DATE, end=END_DATE, progress=False)
    if "Adj Close" in nifty_ref.columns:
        nifty_series = nifty_ref["Adj Close"]
    elif "Close" in nifty_ref.columns:
        nifty_series = nifty_ref["Close"]
    else:
        nifty_series = nifty_ref.select_dtypes("number").iloc[:, 0]
    nifty_ret = nifty_series.pct_change().dropna()

    results = {}
    for sym in adj.columns:
        px = adj[sym]
        ret = returns[sym]
        if ret.isna().sum() > len(ret) * 0.5:
            continue

        cagr = compute_cagr(px)
        ann_vol = ret.std() * np.sqrt(252)
        ann_ret = ret.mean() * 252
        sharpe = (ann_ret - RISK_FREE_RATE) / ann_vol if ann_vol > 0 else np.nan
        down = downside_std(ret) * np.sqrt(252)
        sortino = (ann_ret - RISK_FREE_RATE) / down if down > 0 else np.nan
        roll_max = px.cummax()
        dd = (px / roll_max) - 1
        max_dd = dd.min()
        calmar = ann_ret / abs(max_dd) if max_dd < 0 else np.nan
        common = ret.index.intersection(nifty_ret.index)
        aligned = pd.concat([ret.loc[common], nifty_ret.loc[common]], axis=1).dropna()
        if len(aligned) > 100 and np.var(aligned.iloc[:, 1]) > 0:
            beta = np.cov(aligned.iloc[:, 0], aligned.iloc[:, 1])[0, 1] / np.var(aligned.iloc[:, 1])
        else:
            beta = np.nan
        rec_days = rolling_recovery_days(px)

        results[sym] = {
            "CAGR_10Y": cagr, "Sharpe": sharpe, "Sortino": sortino, "Calmar": calmar,
            "Volatility": ann_vol, "Max_Drawdown": max_dd, "Beta": beta, "Recovery_Days": rec_days
        }
    return pd.DataFrame(results).T

# SCORING SYSTEM
def normalize(v, cap): return max(0, min(v / cap, 1)) if pd.notna(v) else 0
def inverse(v, cap): return max(0, min(1 - v / cap, 1)) if pd.notna(v) else 0
def dd_norm(v): return max(0, min(1 - abs(v) / 0.6, 1)) if pd.notna(v) else 0
def beta_norm(v):
    if pd.isna(v): return 0.5
    v = float(v)
    if v <= 1.2: return 1 - abs(v - 1) / 0.4
    return max(0, 1 - (v - 1.2) / 0.8)

weights = {
    "CAGR_10Y": 25, "Sharpe": 15, "Sortino": 10, "Calmar": 10,
    "Volatility": 10, "Max_Drawdown": 10, "Beta": 5, "Recovery_Days": 5
}

def score(row):
    s = 0
    s += normalize(row["CAGR_10Y"], 0.20) * weights["CAGR_10Y"]
    s += normalize(row["Sharpe"], 1.5) * weights["Sharpe"]
    s += normalize(row["Sortino"], 2.0) * weights["Sortino"]
    s += normalize(row["Calmar"], 1.5) * weights["Calmar"]
    s += inverse(row["Volatility"], 0.35) * weights["Volatility"]
    s += dd_norm(row["Max_Drawdown"]) * weights["Max_Drawdown"]
    s += beta_norm(row["Beta"]) * weights["Beta"]
    s += inverse(row["Recovery_Days"], 800) * weights["Recovery_Days"]
    return (s / sum(weights.values())) * 100

# RUN BLUECHIP ANALYSIS
adj = download_adj_close(BLUECHIP)

if adj.empty:
    print(f"\n⚠️ No data found for Blue-Chip stocks.")
else:
    metrics = calculate_metrics(adj)
    metrics["Decision_Score"] = metrics.apply(score, axis=1)
    top5 = metrics.sort_values("Decision_Score", ascending=False).head(5)

    print("\n📈 Analysis Complete!")
    print("💡 Top 5 Blue-Chip Stocks for Beginner Investors (10-Year View):\n")

    for i, (sym, row) in enumerate(top5.iterrows(), start=1):
        print(f"🏅 {i}. {sym}")
        print(f"   📊 Overall Score (Safety + Growth): {row['Decision_Score']:.2f}/100")
        print(f"   💰 CAGR (10Y): {row['CAGR_10Y']*100:,.2f}%")
        print(f"   📈 Sharpe: {row['Sharpe']:.2f} | Sortino: {row['Sortino']:.2f} | Calmar: {row['Calmar']:.2f}")
        print(f"   🛡️ Volatility: {row['Volatility']*100:,.2f}% | Max DD: {row['Max_Drawdown']*100:,.2f}% | Beta: {row['Beta']:.2f}")
        print(f"   🕒 Recovery Days: {row['Recovery_Days']:.0f} days")
        print("-" * 70)

    print("\n✅ Interpretation:")
    print("→ Higher CAGR = stronger long-term growth.")
    print("→ Sharpe > 0.5 = steady, risk-adjusted returns.")
    print("→ Lower Volatility & Drawdown = safer investment.")
    print("→ Beta < 1 means stock moves less than market (safer).")
    print("→ Lower Recovery Days = quicker bounce-back after dips.")
    print("\n🧾 Explanation of Terms:")
    print("• CAGR (Compound Annual Growth Rate): Average yearly growth of stock price.")
    print("• Sharpe Ratio: Measures how much return you earn for the risk you take. Higher = better.")
    print("• Sortino Ratio: Like Sharpe, but only counts downside (bad) volatility. Higher = safer returns.")
    print("• Calmar Ratio: Compares yearly return vs biggest loss. Higher = strong recovery ability.")
    print("• Volatility: How much the stock price moves up or down. Lower = more stable.")
    print("• Max Drawdown: Biggest price fall from peak to bottom. Lower = safer.")
    print("• Beta: How much the stock moves compared to market (1 = same as market, <1 = safer).")
    print("• Recovery Days: Time it takes for stock to recover after big fall. Fewer = better.")
    print("• Decision Score: Final 0–100 score combining all metrics for beginner investors.")


📥 Downloading 48 Blue-Chip tickers... This may take a few minutes.

📈 Analysis Complete!
💡 Top 5 Blue-Chip Stocks for Beginner Investors (10-Year View):

🏅 1. TITAN
   📊 Overall Score (Safety + Growth): 59.19/100
   💰 CAGR (10Y): 27.52%
   📈 Sharpe: 0.82 | Sortino: 1.21 | Calmar: 0.70
   🛡️ Volatility: 29.61% | Max DD: -41.73% | Beta: 0.91
   🕒 Recovery Days: 652 days
----------------------------------------------------------------------
🏅 2. RELIANCE
   📊 Overall Score (Safety + Growth): 56.91/100
   💰 CAGR (10Y): 22.17%
   📈 Sharpe: 0.70 | Sortino: 1.05 | Calmar: 0.53
   🛡️ Volatility: 27.09% | Max DD: -45.09% | Beta: 1.09
   🕒 Recovery Days: 519 days
----------------------------------------------------------------------
🏅 3. TATACONSUM
   📊 Overall Score (Safety + Growth): 56.86/100
   💰 CAGR (10Y): 25.34%
   📈 Sharpe: 0.75 | Sortino: 1.12 | Calmar: 0.62
   🛡️ Volatility: 30.03% | Max DD: -44.30% | Beta: 0.93
   🕒 Recovery Days: 685 days
--------------------------------------------